In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utils_nn as utlnn

In [3]:
x_test = pd.read_csv("./Train_Test/x_test.csv")
y_test = pd.read_csv("./Train_Test/y_test.csv")

In [4]:
autoencoder_Lat_Lon = tf.keras.models.load_model('modelos_entrenamiento/entrenamiento_lat_lon2.keras')
autoencoder_Height = tf.keras.models.load_model('modelos_entrenamiento/entrenamiento_alturas2.keras')

In [ ]:
#lat_pred,lon_pred = autoencoder_Lat_Lon.predict(x_test)

#MM TENGO QUE MODIFICAR MI CODIGO

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


ValueError: too many values to unpack (expected 2)